In [1]:
from datetime import datetime, timedelta
import requests
import pickle
from pathlib import Path

# Exploration

In [2]:
## copied straight from article
def get_user_reviews(review_appid, params):

    user_review_url = f'https://store.steampowered.com/appreviews/{review_appid}'
    req_user_review = requests.get(
        user_review_url,
        params=params
    )

    if req_user_review.status_code != 200:
        print(f'Fail to get response. Status code: {req_user_review.status_code}')
        return {"success": 2}

    try:
        user_reviews = req_user_review.json()
    except:
        return {"success": 2}

    return user_reviews

In [3]:
review_appid = 1245620
params = {
    'json':1,
    'language': 'english',
    'cursor': '*',                                  # set the cursor to retrieve reviews from a specific "page"
    'num_per_page': 100,
    'filter': 'recent'
}

reviews_response = get_user_reviews(review_appid, params)

In [4]:
reviews_response.keys()

dict_keys(['success', 'query_summary', 'reviews', 'cursor'])

# Write Reviews for One Game

In [5]:
## adapted from article
def get_reviews_for_gameID(appid, review_count_cutoff=1000, datetime_cutoff=datetime(2024, 1, 1, 0, 0, 0)):
    
    # the params of the API
    params = {
        'json':1,
        'language': 'english',
        'cursor': '*',                                  # set the cursor to retrieve reviews from a specific "page"
        'num_per_page': 100,
        'filter': 'recent'
    }
    
    # time_interval = timedelta(hours=24)                         # the time interval to get the reviews
    # end_time = datetime.fromtimestamp(1716718910)               # the timestamp in the return result are unix timestamp (GMT+0)
    end_time = datetime.now()
    # start_time = end_time - time_interval
    start_time = datetime_cutoff
    
    print(f"Start time: {start_time}")
    print(f"End time: {end_time}")
    print(start_time.timestamp(), end_time.timestamp())
    
    passed_start_time = False
    passed_end_time = False
    
    selected_reviews = []

    ## Run this loop until we get a certain number of reviews, or until timestamps are outside of our time bounds
    while (len(selected_reviews) < review_count_cutoff) and (not passed_start_time or not passed_end_time):
    
        reviews_response = get_user_reviews(appid, params)
    
        # not success?
        if reviews_response["success"] != 1:
            print("Not a success")
            print(reviews_response)
    
        if reviews_response["query_summary"]['num_reviews'] == 0:
            print("No reviews.")
            print(reviews_response)
    
        for review in reviews_response["reviews"]:
            recommendation_id = review['recommendationid']
    
            timestamp_created = review['timestamp_created']
            timestamp_updated = review['timestamp_updated']
    
            # skip the comments that beyond end_time
            if not passed_end_time:
                if timestamp_created > end_time.timestamp():
                    continue
                else:
                    passed_end_time = True
    
            # exit the loop once detected a comment that before start_time
            if not passed_start_time:
                if timestamp_created < start_time.timestamp():
                    passed_start_time = True
                    break
    
            # extract the useful (to me) data
            ## these attributes are taken right from the article
            ## we could see what other useful info there is
            
            author_steamid = review['author']['steamid']        # will automatically redirect to the profileURL if any
            playtime_forever = review['author']['playtime_forever']
            playtime_last_two_weeks = review['author']['playtime_last_two_weeks']
            playtime_at_review_minutes = review['author']['playtime_at_review']
            last_played = review['author']['last_played']
    
            review_text = review['review']
            voted_up = review['voted_up']
            votes_up = review['votes_up']
            votes_funny = review['votes_funny']
            weighted_vote_score = review['weighted_vote_score']
            steam_purchase = review['steam_purchase']
            received_for_free = review['received_for_free']
            written_during_early_access = review['written_during_early_access']
    
            my_review_dict = {
                'recommendationid': recommendation_id,
                'author_steamid': author_steamid,
                'playtime_at_review_minutes': playtime_at_review_minutes,
                'playtime_forever_minutes': playtime_forever,
                'playtime_last_two_weeks_minutes': playtime_last_two_weeks,
                'last_played': last_played,
    
                'review_text': review_text,
                'timestamp_created': timestamp_created,
                'timestamp_updated': timestamp_updated,
    
                'voted_up': voted_up,
                'votes_up': votes_up,
                'votes_funny': votes_funny,
                'weighted_vote_score': weighted_vote_score,
                'steam_purchase': steam_purchase,
                'received_for_free': received_for_free,
                'written_during_early_access': written_during_early_access,
            }

            ## store results as list of dicts, one dict per review
            selected_reviews.append(my_review_dict)
    
        # go to next page
        try:
            cursor = reviews_response['cursor']         # cursor field does not exist in the last page
        except Exception as e:
            cursor = ''
    
        # no next page
        # exit the loop
        if not cursor:
            print("Reached the end of all comments.")
            break
    
        # set the cursor object to move to next page to continue
        params['cursor'] = cursor
        print('To next page. Next page cursor:', cursor)
    return selected_reviews

In [6]:
# test get_reviews function
reviews = get_reviews_for_gameID(1245620)
len(reviews)

Start time: 2024-01-01 00:00:00
End time: 2025-11-16 18:49:44.886125
1704096000.0 1763347784.886125
To next page. Next page cursor: AoJ43tCwxJoDdM+6ngY=
To next page. Next page cursor: AoJ47audwJoDdaX1nQY=
To next page. Next page cursor: AoJ4xez/upoDfYSlnQY=
To next page. Next page cursor: AoJwrti3t5oDdtfwnAY=
To next page. Next page cursor: AoJwhYuAs5oDdNSunAY=
To next page. Next page cursor: AoJ4yKGSr5oDd5bsmwY=
To next page. Next page cursor: AoJ4zc7JqpoDfaGrmwY=
To next page. Next page cursor: AoJw5qqKpJoDdt3JmgY=
To next page. Next page cursor: AoJw24PDn5oDfu/1mQY=
To next page. Next page cursor: AoJw5evNmZoDf4eYmQY=


1000

In [32]:
## my own doing
def review_to_csv(appname, appid, review_count_cutoff=1000, datetime_cutoff=datetime(2024, 1, 1, 0, 0, 0), save_to_csv=True):
    reviews = get_reviews_for_gameID(appid, review_count_cutoff, datetime_cutoff)
    review_df = pd.DataFrame(reviews)
    review_df["Appname"] = appname ## add columne for game name
    review_df = review_df.set_index("recommendationid") ## set rec ID to be the index
    ## should confirm what happens to this index when we stack multiple games

    ## If save to csv is on, write results to csv in its own folder
    if save_to_csv:
        appname_cleaned = appname.replace(" ", "_")
        foldername = f"{appid}_{appname_cleaned}"
        filename = f"{appid}_{appname_cleaned}_reviews.csv"
        output_path = Path(
            foldername, filename
        )
        if not output_path.parent.exists():
            output_path.parent.mkdir(parents=True)

        print(f"Writing results for {appname} to {output_path}")
        review_df.to_csv(output_path)

    ## whether or not we write results out, still return the DF object
    return review_df
    

In [34]:
## test the function
review_df = review_to_csv("Elden Ring", 1245620)
review_df

Start time: 2024-01-01 00:00:00
End time: 2025-11-16 19:02:24.650646
1704096000.0 1763348544.650646
To next page. Next page cursor: AoJ43tCwxJoDdM+6ngY=
To next page. Next page cursor: AoJ47audwJoDdaX1nQY=
To next page. Next page cursor: AoJ4xez/upoDfYSlnQY=
To next page. Next page cursor: AoJwrti3t5oDdtfwnAY=
To next page. Next page cursor: AoJwhYuAs5oDdNSunAY=
To next page. Next page cursor: AoJ4yKGSr5oDd5bsmwY=
To next page. Next page cursor: AoJ4zc7JqpoDfaGrmwY=
To next page. Next page cursor: AoJw5qqKpJoDdt3JmgY=
To next page. Next page cursor: AoJw24PDn5oDfu/1mQY=
To next page. Next page cursor: AoJw5evNmZoDf4eYmQY=
Writing results for Elden Ring to 1245620_Elden Ring/1245620_Elden Ring_reviews.csv


,author_steamid,playtime_at_review_minutes,playtime_forever_minutes,playtime_last_two_weeks_minutes,last_played,review_text,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,steam_purchase,received_for_free,written_during_early_access,Appname
recommendationid,,,,,,,,,,,,,,,,
209438568,76561199086155355,28814,28814,0,1756702299,its alright,1763342403,1763342403,True,0,0,0.5,True,False,False,Elden Ring
209438358,76561199828725549,605,605,605,1763324952,ITS CINEMA,1763342174,1763342174,True,0,0,0.5,True,False,False,Elden Ring
209437238,76561199384098710,730,730,730,1763275159,"i rekomend, very cool buss fights, very explor...",1763340819,1763340819,True,0,0,0.5,True,False,False,Elden Ring
209437134,76561198994193106,16697,16697,0,1729997510,"Amazing, all that needs to be said.",1763340712,1763340712,True,0,0,0.5,True,False,False,Elden Ring
209434465,76561199576978800,1745,1745,1745,1763336800,"In the Lands Between, glory is not granted… it...",1763337456,1763337456,True,0,0,0.5,True,False,False,Elden Ring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207933017,76561199861948706,6663,8753,2908,1762831592,I died in the main menu screen,1761801139,1762392402,True,0,0,0.5,True,False,False,Elden Ring
207932630,76561198981863098,5532,6447,1068,1762919954,malenia speedrun,1761800533,1762222799,True,0,0,0.5,True,False,False,Elden Ring
207932223,76561198207225744,1126,1627,529,1762454566,10/10 so far\ntook me longer to defeat Margit ...,1761799852,1762191002,True,1,0,0.523809552192687988,True,False,False,Elden Ring


In [35]:
## Read csv back in and confirm it looks okay
appname = "Elden Ring"
appid = 1245620


review_path = f"{appid}_{appname}/{appid}_{appname}_reviews.csv"

review_df_csv = pd.read_csv(review_path, index_col=0)
review_df_csv

,author_steamid,playtime_at_review_minutes,playtime_forever_minutes,playtime_last_two_weeks_minutes,last_played,review_text,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,steam_purchase,received_for_free,written_during_early_access,Appname
recommendationid,,,,,,,,,,,,,,,,
209438568,76561199086155355,28814,28814,0,1756702299,its alright,1763342403,1763342403,True,0,0,0.50000,True,False,False,Elden Ring
209438358,76561199828725549,605,605,605,1763324952,ITS CINEMA,1763342174,1763342174,True,0,0,0.50000,True,False,False,Elden Ring
209437238,76561199384098710,730,730,730,1763275159,"i rekomend, very cool buss fights, very explor...",1763340819,1763340819,True,0,0,0.50000,True,False,False,Elden Ring
209437134,76561198994193106,16697,16697,0,1729997510,"Amazing, all that needs to be said.",1763340712,1763340712,True,0,0,0.50000,True,False,False,Elden Ring
209434465,76561199576978800,1745,1745,1745,1763336800,"In the Lands Between, glory is not granted… it...",1763337456,1763337456,True,0,0,0.50000,True,False,False,Elden Ring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207933017,76561199861948706,6663,8753,2908,1762831592,I died in the main menu screen,1761801139,1762392402,True,0,0,0.50000,True,False,False,Elden Ring
207932630,76561198981863098,5532,6447,1068,1762919954,malenia speedrun,1761800533,1762222799,True,0,0,0.50000,True,False,False,Elden Ring
207932223,76561198207225744,1126,1627,529,1762454566,10/10 so far\ntook me longer to defeat Margit ...,1761799852,1762191002,True,1,0,0.52381,True,False,False,Elden Ring


# Scrape Reviews for all games in sample

For next time, or one of you guys to pick up... we need to determine our sample of games somehow